<a href="https://colab.research.google.com/github/siddharthtelang/Face-Detection/blob/main/Face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
!ls -la 'drive/MyDrive/Colab Notebooks/data.mat'

-rw------- 1 root root 2419392 Oct 30 21:11 'drive/MyDrive/Colab Notebooks/data.mat'


In [3]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.decomposition import PCA

In [4]:
data = sio.loadmat('drive/MyDrive/Colab Notebooks/data.mat')

In [5]:
faces = data.get('face')

In [6]:
faces.shape

(24, 21, 600)

In [7]:
# init the training and testing datad
total_subjects = 200
training_size = int((2/3)*faces.shape[2])
testing_size = int((1/3)*faces.shape[2])
original_image_size = faces.shape[0]*faces.shape[1]
# training_data = np.zeros(shape=(training_size, faces.shape[0]*faces.shape[1]))
# testing_data = np.zeros(shape=(testing_size, faces.shape[0]*faces.shape[1]))
training_data= []
testing_data = []

In [8]:
c = 1 # counter to pick random sample from either illumination or 
train_index = 0
test_index = 0
for i in range(0, faces.shape[2], 3):
    training_data.append(faces[:,:,i].flatten())
    if c % 2 == 0:
      training_data.append(faces[:,:,i+1].flatten())
      testing_data.append(faces[:,:,i+2].flatten())
    else:
      training_data.append(faces[:,:,i+2].flatten())
      testing_data.append(faces[:,:,i+1].flatten())
    c += 1

In [13]:
fig, ax_array = plt.subplots(nrows=40, ncols=10, figsize=(20,20))

# flatten the array of axes, which makes them easier to iterate through and assign
ax_array = ax_array.flatten()

# plot the faces
for i, ax in enumerate(ax_array):
    ax.imshow(training_data[i].reshape(24,21))

Error in callback <function flush_figures at 0x7f86911e5320> (for post_execute):


KeyboardInterrupt: ignored

In [9]:
mu = []
cov = []
for i in range(0, len(training_data), 2):
    mean = ((training_data[i] + training_data[i+1]) / 2).reshape(1, original_image_size)
    cov1 = np.matmul((training_data[i]-mean).T, training_data[i]-mean)
    cov2 = np.matmul((training_data[i+1]-mean).T, training_data[i+1]-mean)
    noise = 0.24*np.identity(cov1.shape[0])
    cov_ = (cov1 + cov2)/2 + noise
    cov.append(cov_)
    mu.append(mean)
    if np.linalg.det(cov_) == 0 or np.linalg.det(cov_) == 0.0:
        print('alert - zero determinant')

PCA


In [15]:
check = np.zeros(shape=(600, faces.shape[0]*faces.shape[1]))
print(check.shape)

for i in range(600):
    temp = faces[:,:,i]
    check[i] = temp.flatten()

pca = PCA(400)
projected = pca.fit_transform(check)
print(projected.shape)

(600, 504)
(600, 400)


In [16]:
usePCA = True
if (usePCA):
    training_data.clear()
    testing_data.clear()
    c = 1 # counter to pick random sample from either illumination or 
    for i in range(0, faces.shape[2], 3):
        training_data.append(projected[i+2])
        if c % 2 == 0:
          training_data.append(projected[i])
          testing_data.append(projected[i+1])
        else:
            training_data.append(projected[i+1])
            testing_data.append(projected[i])
        # c += 1

In [17]:
if(usePCA):
    mu = []
    cov = []
    for i in range(0, len(training_data), 2):
        mean = ((training_data[i] + training_data[i+1]) / 2).reshape(1, projected.shape[1])
        cov1 = np.matmul((training_data[i]-mean).T, training_data[i]-mean)
        cov2 = np.matmul((training_data[i+1]-mean).T, training_data[i+1]-mean)
        noise = 0.24*np.identity(cov1.shape[0])
        cov_ = (cov1 + cov2)/2 + noise
        cov.append(cov_)
        mu.append(mean)
        if np.linalg.det(cov_) == 0 or np.linalg.det(cov_) == 0.0:
            print('alert - zero determinant')
    

In [18]:
score = 0
for i in range(testing_size):
    likelihood_list = []
    for j in range(testing_size):
        likelihood = (-0.5)*math.log(np.linalg.det(cov[j])) - (0.5)*np.dot( testing_data[i]-mu[j], np.dot( np.linalg.inv(cov[j]), (testing_data[i]-mu[j]).T ) )
        likelihood_list.append(likelihood)
    temp = np.array(likelihood_list)
    if np.argmax(temp) == i:
        score += 1
        print('Correct, score is now ', score)
    else:
      print('Incorrect Score for subject ', i)
    # break

Correct, score is now  1
Correct, score is now  2
Correct, score is now  3
Correct, score is now  4
Correct, score is now  5
Correct, score is now  6
Correct, score is now  7
Correct, score is now  8
Correct, score is now  9
Correct, score is now  10
Correct, score is now  11
Correct, score is now  12
Incorrect Score for subject  12
Incorrect Score for subject  13
Incorrect Score for subject  14
Correct, score is now  13
Correct, score is now  14
Correct, score is now  15
Correct, score is now  16
Correct, score is now  17
Incorrect Score for subject  20
Correct, score is now  18
Correct, score is now  19
Correct, score is now  20
Correct, score is now  21
Correct, score is now  22
Correct, score is now  23
Correct, score is now  24
Incorrect Score for subject  28
Correct, score is now  25
Correct, score is now  26
Correct, score is now  27
Correct, score is now  28
Incorrect Score for subject  33
Correct, score is now  29
Correct, score is now  30
Incorrect Score for subject  36
Incor

In [119]:
print("Accuracy of Bayes Classifier for Subjects with ML = " , (100*score/total_subjects))

Accuracy of Bayes Classifier for Subjects with ML =  72.0


In [17]:
score_list = []


NameError: ignored

In [118]:
score_list.append(score)

NameError: ignored

In [120]:
from google.colab import files
uploaded = files.upload()


Saving pose.mat to pose.mat
